# Exercises for "Hands-on with Pydata: How to Build a Minimal Recommendation Engine"

# Systems check: imports and files

In [1]:
import numpy as np
import pandas as pd

# Pandas questions: Series and DataFrames
## 1. Adding a column in a DataFrame

In [3]:
# given the following DataFrame, add a new column to it
df = pd.DataFrame({'col1': [1,2,3,4]})
# adding column 2
df['col2'] = [5,6,7,8]
df

,col1,col2
0,1,5
1,2,6
2,3,7
3,4,8


## 2. Deleting a row in a DataFrame

In [6]:
# given the following DataFrame, delete row 'd' from it
df = pd.DataFrame({'col1': [1,2,3,4]}, index = ['a','b','c','d'])
df = df.drop('d', axis=0)
df

,col1
a,1
b,2
c,3


## 3. Creating a DataFrame from a few Series

In [7]:
# given the following three Series, create a DataFrame such that it holds them as its columns
ser_1 = pd.Series(np.random.randn(6))
ser_2 = pd.Series(np.random.randn(6))
ser_3 = pd.Series(np.random.randn(6))

df1 = pd.DataFrame({'col1': ser_1, 'col2': ser_2, 'col3': ser_3})
df1

,col1,col2,col3
0,0.217001,-0.112411,-0.735951
1,-2.094599,-1.911710,0.279573
2,1.593549,0.189199,2.153321
3,-0.271091,-0.437601,1.115909
4,-0.963317,-0.515059,-1.215597
5,0.156287,-0.691717,-1.452506


# Pandas questions: Indexing

## 1. Indexing into a specific column

In [10]:
# given the following DataFrame, try to index into the 'col_2' column
df = pd.DataFrame(data={'col_1': [0.12, 7, 45, 10], 'col_2': [0.9, 9, 34, 11]},
                  columns=['col_1', 'col_2', 'col_3'],
                  index=['obs1', 'obs2', 'obs3', 'obs4'])
df['col_2']

obs1     0.9
obs2     9.0
obs3    34.0
obs4    11.0
Name: col_2, dtype: float64

## 2. Label-based indexing

In [11]:
# using the same DataFrame, index into the row whose index is 'obs3'
df.loc['obs3']

col_1     45
col_2     34
col_3    NaN
Name: obs3, dtype: object

## 2. Position-based indexing

In [12]:
# using the same DataFrame, index into into its first row
df.iloc[0]

col_1    0.12
col_2     0.9
col_3     NaN
Name: obs1, dtype: object

# Mini-Challenge prep: data loading

## 1. How to load the `users` and `movies` portions of MovieLens

In [14]:
import pandas as pd

users = pd.read_table('data/ml-1m/users.dat',
                      sep='::', header=None,
                      names=['user_id', 'gender', 'age', 'occupation', 'zip'], engine='python')

movies = pd.read_table('data/ml-1m/movies.dat',
                       sep='::', header=None,
                       names=['movie_id', 'title', 'genres'], engine='python')

## 2. How to load the training and testing subsets

In [15]:
# subset version (hosted notebook)
movielens_train = pd.read_csv('data/movielens_train.csv', index_col=0)
movielens_test = pd.read_csv('data/movielens_test.csv', index_col=0)

In [16]:
movielens_train.head()

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres,for_testing
593263,3562,3798,4,967332344,F,25,6,32812,What Lies Beneath (2000),Thriller,False
235597,1051,3793,4,974958593,F,25,0,60513,X-Men (2000),Action|Sci-Fi,False
219003,3727,2366,3,966309522,M,35,7,74401,King Kong (1933),Action|Adventure|Horror,False
685090,4666,1094,3,963843918,M,35,1,53704,"Crying Game, The (1992)",Drama|Romance|War,False
312377,3261,1095,4,968251750,M,45,20,87505,Glengarry Glen Ross (1992),Drama,False


In [92]:
users.loc[users.user_id == 3948]

,user_id,gender,age,occupation,zip
3947,3948,F,45,12,61554


In [17]:
movielens_test.head()

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres,for_testing
693323,4653,2648,4,975532459,M,35,12,95051,Frankenstein (1931),Horror,False
24177,2259,1270,4,974591524,F,56,16,70503,Back to the Future (1985),Comedy|Sci-Fi,False
202202,3032,1378,5,970343147,M,25,0,47303,Young Guns (1988),Action|Comedy|Western,False
262003,3029,2289,4,972846393,M,18,4,92037,"Player, The (1992)",Comedy|Drama,False
777848,4186,2403,3,1017931262,M,25,7,33308,First Blood (1982),Action,False


# Mini-Challenge prep: evaluation functions

These are the two functions that you will need to test your `estimate` method.

In [18]:
def compute_rmse(y_pred, y_true):
    """ Compute Root Mean Squared Error. """
    
    return np.sqrt(np.mean(np.power(y_pred - y_true, 2)))

In [102]:
def evaluate(estimate_f):
    """ RMSE-based predictive performance evaluation with pandas. """
    
    ids_to_estimate = zip(movielens_test.user_id, movielens_test.movie_id)
    estimated = np.array([estimate_f(u,i) for (u,i) in ids_to_estimate])
    real = movielens_test.rating.values
    return compute_rmse(estimated, real)

    print (ids_to_estimate)

Test a dummy solution!

In [20]:
def my_estimate_func(user_id, movie_id):
    return 3.0

You can test for performance with the following line, which assumes that your function is called `my_estimate_func`:

In [21]:
print 'RMSE for my estimate function: %s' % evaluate(my_estimate_func)

RMSE for my estimate function: 1.23237195265


# Reco systems questions: Minimal reco engine v1.0

## 1. Simple collaborative filtering using mean ratings

In [31]:
# write an 'estimate' function that computes the mean rating of a particular user

def collab_mean(user_id, movie_id):
    # index into all ratings of this movie
    item_condition = movielens_train.movie_id == movie_id
    # select all ratings except the user's 
    user_condition = movielens_train.user_id != user_id
    # compute the ratings
    everyone_else_ratings = movielens_train.loc[item_condition & user_condition, 'rating']
    
    if everyone_else_ratings.empty:
        return 3.0
    else:
        return everyone_else_ratings.mean()

    
# try it out for a user_id, movie_id pair
print collab_mean(4653, 2648)
print 'RMSE for estimate2: %s' % evaluate(collab_mean)

4.0
RMSE for estimate2: 1.1234279896


# Mini-Challenge: first round
Implement an `estimate` function of your own using other similarity notions, eg.:

- collaborative filter based on age similarities
- collaborative filter based on zip code similarities
- collaborative filter based on occupation similarities
- content filter based on movie genre

In [41]:
# trying my own estimate function based on weighted average of different similarities

w1 = 0.4
w2 = 0.6

def my_funct(user_id, movie_id):
    # index into all ratings of this movie
    item_condition = movielens_train.movie_id == movie_id
    # select all ratings except the user's 
    user_condition = movielens_train.user_id != user_id
    # access ratings
    r1 = movielens_train.loc[item_condition].rating 
    r2 = movielens_train.loc[user_condition].rating
    # compute the weighted average of means or fall back to overall mean
    if (r1.empty or r2.empty):
        return movielens_train.rating.mean()
    else:
        return w1 * r1.mean() + w2 * r2.mean()

    
# try it out for a user_id, movie_id pair
print my_funct(4653, 2648)
print 'RMSE for estimate3: %s' % evaluate(my_funct)

3.72740359897
RMSE for estimate3: 1.04787655142


This is a better result than obtained in the tutorial (1.0578)!

In [128]:
# define new data frame to access movie genre
moviegenres = movies[['movie_id', 'genres']].set_index('movie_id')
moviegenres.loc[2640:2650]

,genres
movie_id,
2640,Action|Adventure|Sci-Fi
2641,Action|Adventure|Sci-Fi
2642,Action|Adventure|Sci-Fi
2643,Action|Adventure|Sci-Fi
2644,Horror
2645,Horror
2646,Horror
2647,Horror
2648,Horror


In [131]:
# try to improve result by introducing age, zip code and occupation in the user condition

w1 = 0.3
w2 = 0.7

def my_funct_2(user_id, movie_id):
    # select movies of the same genre rated by the user in question - content filter
    item_condition = np.logical_and(movielens_train.user_id == user_id, moviegenres.loc[movie_id].genres == movielens_train.genres)
    # select users of same gender, occupation and age group - collaborative filter
    user_condition = np.logical_and((np.logical_and(movielens_train.gender == users.loc[user_id].gender,  
                      movielens_train.occupation == users.loc[user_id].occupation)), 
                                   abs(movielens_train.age - users.loc[user_id].age) < 10)
    # compute the mean of those ratings
    r1 = movielens_train.loc[item_condition].rating
    r2 = movielens_train.loc[user_condition].rating
    
    if (r1.empty or r2.empty):
        return movielens_train.rating.mean()
    else:
        return w1 * r1.mean() + w2 * r2.mean()

    
# try it out for a user_id, movie_id pair
print my_funct(4653, 2648)
print 'RMSE for estimate3: %s' % evaluate(my_funct_2)

3.5457348407
RMSE for estimate3: 1.08400057665


After testing with different weights, this does not give a better result than simpler function above.

# Mini-Challenge: second round
Implement an `estimate` function of your own using other custom similarity notions, eg.:

- euclidean
- cosine

In [138]:
def pearson(s1, s2):
    """Take two pd.Series objects and return a pearson correlation."""
    s1_c = s1 - s1.mean()
    s2_c = s2 - s2.mean()
    return np.sum(s1_c * s2_c) / np.sqrt(np.sum(s1_c ** 2) * np.sum(s2_c ** 2))

In [139]:
def euclidean(s1, s2):
    """Take two pd.Series objects and return their euclidean 'similarity'."""
    diff = s1 - s2
    return 1 / (1 + np.sqrt(np.sum(diff ** 2)))

In [140]:
def cosine(s1, s2):
    """Take two pd.Series objects and return their cosine similarity."""
    return np.sum(s1 * s2) / np.sqrt(np.sum(s1 ** 2) * np.sum(s2 ** 2))

In [135]:
# pearson correlation similarity function using simplest user and movie conditions

class CollabPearsonReco:
    """ Collaborative filtering using a custom sim(u,u'). """

    def learn(self):
        """ Prepare datastructures for estimation. """
        
        self.all_user_profiles = movielens_train.pivot_table('rating', index='movie_id', columns='user_id')

    def estimate(self, user_id, movie_id):
        """ Ratings weighted by correlation similarity. """
                
        user_condition = movielens_train.user_id != user_id
        movie_condition = movielens_train.movie_id == movie_id
        ratings_by_others = movielens_train.loc[user_condition & movie_condition]
        if ratings_by_others.empty: 
            return 3.0
        
        ratings_by_others.set_index('user_id', inplace=True)
        their_ids = ratings_by_others.index
        their_ratings = ratings_by_others.rating
        their_profiles = self.all_user_profiles[their_ids]
        user_profile = self.all_user_profiles[user_id]
        sims = their_profiles.apply(lambda profile: pearson(profile, user_profile), axis=0)
        ratings_sims = pd.DataFrame({'sim': sims, 'rating': their_ratings})
        ratings_sims = ratings_sims[ratings_sims.sim > 0]
        if ratings_sims.empty:
            return their_ratings.mean()
        else:
            return np.average(ratings_sims.rating, weights=ratings_sims.sim)
        
reco = CollabPearsonReco()
reco.learn()
print 'RMSE for CollabPearsonReco: %s' % evaluate(reco.estimate)

C:\Users\Work\Anaconda2\lib\site-packages\ipykernel\__main__.py:5: RuntimeWarning: invalid value encountered in double_scalars


RMSE for CollabPearsonReco: 1.12640340009


In [147]:
# Euclidean similarity finct gives maroginally better results for the same user and movie conditions as above
# this class uses Euclidean funct on more detailed user and movie conditions

class CollabEuclideanReco:
    """ Collaborative filtering using a custom sim(u,u'). """

    def learn(self):
        """ Prepare datastructures for estimation. """
        
        self.all_user_profiles = movielens_train.pivot_table('rating', index='movie_id', columns='user_id')

    def estimate(self, user_id, movie_id):
        """ Ratings weighted by correlation similarity. """
        
        user_condition = np.logical_and((np.logical_and(movielens_train.gender == users.loc[user_id].gender,  
                      movielens_train.occupation == users.loc[user_id].occupation)), 
                                   abs(movielens_train.age - users.loc[user_id].age) < 10)
        movie_condition = np.logical_and(movielens_train.user_id == user_id, moviegenres.loc[movie_id].genres == movielens_train.genres)
        ratings_by_others = movielens_train.loc[user_condition & movie_condition]
        
        #r1 = movielens_train.loc[item_condition].rating 
        #r2 = movielens_train.loc[user_condition].rating
        if ratings_by_others.empty: 
            return movielens_train.rating.mean()
        
        ratings_by_others.set_index('user_id', inplace=True)
        their_ids = ratings_by_others.index
        their_ratings = ratings_by_others.rating
        their_profiles = self.all_user_profiles[their_ids]
        user_profile = self.all_user_profiles[user_id]
        sims = their_profiles.apply(lambda profile: euclidean(profile, user_profile), axis=0)
        ratings_sims = pd.DataFrame({'sim': sims, 'rating': their_ratings})
        ratings_sims = ratings_sims[ratings_sims.sim > 0]
        if ratings_sims.empty:
            return their_ratings.mean()
        else:
            return np.average(ratings_sims.rating, weights=ratings_sims.sim)
        
reco = CollabEuclideanReco()
reco.learn()
print 'RMSE for CollabEuclideanReco: %s' % evaluate(reco.estimate)

RMSE for CollabEuclideanReco: 1.09046050902


In [148]:
# try cosine function on the same user and movie conditions

class CollabCosineReco:
    """ Collaborative filtering using a custom sim(u,u'). """

    def learn(self):
        """ Prepare datastructures for estimation. """
        
        self.all_user_profiles = movielens_train.pivot_table('rating', index='movie_id', columns='user_id')

    def estimate(self, user_id, movie_id):
        """ Ratings weighted by correlation similarity. """
        
        user_condition = np.logical_and((np.logical_and(movielens_train.gender == users.loc[user_id].gender,  
                      movielens_train.occupation == users.loc[user_id].occupation)), 
                                   abs(movielens_train.age - users.loc[user_id].age) < 10)
        movie_condition = np.logical_and(movielens_train.user_id == user_id, moviegenres.loc[movie_id].genres == movielens_train.genres)
        ratings_by_others = movielens_train.loc[user_condition & movie_condition]
        if ratings_by_others.empty: 
            return 3.0
        
        ratings_by_others.set_index('user_id', inplace=True)
        their_ids = ratings_by_others.index
        their_ratings = ratings_by_others.rating
        their_profiles = self.all_user_profiles[their_ids]
        user_profile = self.all_user_profiles[user_id]
        sims = their_profiles.apply(lambda profile: cosine(profile, user_profile), axis=0)
        ratings_sims = pd.DataFrame({'sim': sims, 'rating': their_ratings})
        ratings_sims = ratings_sims[ratings_sims.sim > 0]
        if ratings_sims.empty:
            return their_ratings.mean()
        else:
            return np.average(ratings_sims.rating, weights=ratings_sims.sim)
        
reco = CollabCosineReco()
reco.learn()
print 'RMSE for CollabCosineReco: %s' % evaluate(reco.estimate)

RMSE for CollabCosineReco: 1.23328403267


Euclidean function gives the best results, but still does not beat the previously obtained RMSE of 1.0478 from the simpler weighted average function on simpler user and movie conditions.